In [1]:
import json
import numpy as np
def cat_with(y):
    z = []
    for i in y:
        z.append(' '.join(i)) # 把sentence从list -> str
    z2 = []
    for i in y:
        z2 += i
    # 返回一个sentence的str列表和全部拼接的完整sentences（用于校验）
    return z, z2
def relations_change(offsets, y, rel_type_list, all_sentences, sentences, ner_dict_list):
    z = []
    for i, cur_sentence, offset, ner_dict in zip(y, sentences, offsets, ner_dict_list):
        # [r1, r2...]
        zz = [] # 界定不同sentence的relations的边界
        for j in i:
            # r1: [s1, e1, s2, e2, rel_type]
            head_ent_type = ner_dict[str(j[0:2])].lower()
            tail_ent_type = ner_dict[str(j[2:4])].lower()
            jj = [se_value - offset for se_value in j[:-1]] # 将原始的位置减去偏移量得到以当前sentence为起点的坐标

            # 校验对应内容
            assert cur_sentence[jj[0]:jj[1]+1] == all_sentences[j[0]:j[1]+1] and cur_sentence[jj[2]:jj[3]+1] == all_sentences[j[2]:j[3]+1],f'{cur_sentence[jj[0]:jj[1]+1]} == {all_sentences[j[0]:j[1]+1]} and {cur_sentence[jj[2]:jj[3]+1]} == {all_sentences[j[2]:j[3]+1]}'

            jj = [str(jjj) for jjj in jj]
            two_ent_se = ','.join(jj)
            head_ent_se = ','.join(jj[0:2])
            tail_ent_se = ','.join(jj[2:4])
            rel_type = j[-1].lower()

            rel_type_list.add(rel_type)
            rel_type_list.add(head_ent_type)
            rel_type_list.add(tail_ent_type)

            head_ent = head_ent_se + " " + head_ent_type
            tail_ent = tail_ent_se + " " + tail_ent_type
            rel = " " + rel_type
            zz.append(head_ent)
            zz.append(tail_ent)
            zz.append(rel)
        z.append(zz)

    # 返回一个长度和sentence个数一致的relations列表：[[r1,r2],[r3],[r5,r6,r7]...]
    return z, rel_type_list
def change(in_file, out_file, rel_type_list):
    out_data = []
    with open(in_file,"r") as f:
       in_data= f.readlines()
    for idata in in_data:
        idata = json.loads(idata)

        # sentences
        sentences = idata["sentences"]
        offsets = np.cumsum([0] + [len(sentence) for sentence in sentences[:-1]])
        sentences, all_sentences = cat_with(sentences)
        # relations to discontinuous entity
        relations = idata["relations"]

        ner = idata["ner"]
        ner_dict_list = []
        for ner_i in ner:
            ner_dict = {}
            for ner_ii in ner_i:
                if ner_ii == []:
                    continue
                ner_dict[str(ner_ii[:2])] = ner_ii[2]
            ner_dict_list.append(ner_dict)

        relations, rel_type_list = relations_change(offsets, relations, rel_type_list, all_sentences, idata["sentences"], ner_dict_list)

        # 保证sentences、relations个数一致
        assert len(sentences) == len(relations) == len(idata["relations"])
        for r, ro in zip(relations, idata["relations"]):
            assert len(r) == 3*len(ro)
        
        for sentence, relation, ner_i, offset in zip(sentences, relations, ner, offsets):
            out_data.append(sentence)
            ner_i_str = []
            for ner_ii in ner_i:
                ner_ii[0:2] -= offset
                ner_ii = [str(x) for x in ner_ii]
                x = ','.join(ner_ii[0:2]) + " " + ner_ii[2].lower()
                ner_i_str.append(x)
            out_data.append('|'.join(ner_i_str))
            out_data.append('|'.join(relation))
            out_data.append("")
    with open(out_file,"w") as f:
       for i in out_data:
           f.write(i+"\n")
    return rel_type_list

In [2]:
rel_type_list = set()
print(rel_type_list)
rel_type_list = change("/disk1/wxl/Desktop/DeepKE/example/ner/BiSPN/data/ace2005/train.json","//disk1/wxl/Desktop/DeepKE/example/baseline/BARTNER/data/re_ner_ace05/train.txt", rel_type_list)
print(rel_type_list)
rel_type_list = change("/disk1/wxl/Desktop/DeepKE/example/ner/BiSPN/data/ace2005/dev.json","/disk1/wxl/Desktop/DeepKE/example/baseline/BARTNER/data/re_ner_ace05/dev.txt", rel_type_list)
print(rel_type_list)
rel_type_list = change("/disk1/wxl/Desktop/DeepKE/example/ner/BiSPN/data/ace2005/test.json","/disk1/wxl/Desktop/DeepKE/example/baseline/BARTNER/data/re_ner_ace05/test.txt",rel_type_list)
print(rel_type_list)

set()
{'veh', 'part-whole', 'per-soc', 'gen-aff', 'org-aff', 'org', 'loc', 'art', 'wea', 'fac', 'gpe', 'phys', 'per'}
{'veh', 'part-whole', 'per-soc', 'gen-aff', 'org-aff', 'org', 'loc', 'art', 'wea', 'fac', 'gpe', 'phys', 'per'}
{'veh', 'part-whole', 'per-soc', 'gen-aff', 'org-aff', 'org', 'loc', 'art', 'wea', 'fac', 'gpe', 'phys', 'per'}
